In [2]:
import hydra
import cv2 as cv
import numpy as np

from finger_drawing.hand_detection_models import build_hand_detection_model

In [3]:
from omegaconf import OmegaConf


with hydra.initialize("../configs", version_base=None):
    cfg = hydra.compose("config", overrides=["model=media_pipe"])
    hdm = build_hand_detection_model(cfg)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
video_capture = cv.VideoCapture(0)
height = int(video_capture.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(video_capture.get(cv.CAP_PROP_FRAME_WIDTH))
draw_frame = np.zeros((height, width)).astype(np.uint8)
while True:
    ret, frame = video_capture.read()
    if (not ret) | (cv.waitKey(25) & 0xFF == ord("q")):
        break
    frame = frame[:, ::-1, :]
    landmarks = hdm.get_hand_box(frame[:, ::-1, :])
    frame = frame[:, ::-1, :]
    for i, landmark in enumerate(landmarks):
        if i == 8:
            if landmark[0] < 30:
                draw_frame = np.zeros((height, width)).astype(np.uint8)
            draw_frame = cv.circle(
                draw_frame,
                tuple(landmark),
                4,
                color=(255, 255, 255),
                thickness=3
            )
            frame = cv.putText(
                frame,
                str(i),
                tuple(landmark),
                fontFace=1,
                fontScale=1,
                color=(255, 255, 255)
            )
    cv.imshow("Draw", draw_frame)
    cv.imshow("Output", frame)
video_capture.release()
cv.destroyAllWindows()